In [1]:
import numpy as np
import re
import pandas as pd
import matplotlib.pyplot as plt
import h5py

In [2]:
with open("HOF_RHF.out") as f:
    all_lines = f.readlines()

In [3]:
type(all_lines)

list

### Largest coefficients of the exci roots

In [4]:
pat1 = re.compile(r"The Final eigenvalues from the Davidson method")
pat2 = re.compile(r"================...")
pat3 = re.compile(r"Root:...")
pat4 = re.compile(r"C^2 :...")
pat5 = re.compile(r"Root: 0...")

In [5]:
def ex_ene_parse(pat3, all_lines):
    root_ene = []
    for line in all_lines:
        for match in re.finditer(pat3,line.strip()):
            root_ene.append(float(line.strip().split()[4]))
    return root_ene

In [6]:
del_ex_root_ene = ex_ene_parse(pat3,all_lines)
del_ex_root_ene

[0.1744080457739137,
 0.2535162395738729,
 0.3483169930429212,
 0.3821066412378998,
 0.4030583583492398,
 0.486008501509091,
 0.5143698041793688,
 0.5893012916740668]

In [7]:
def from_to(line):
    from_state = line.strip().split()[2]
    to_state = line.strip().split()[0]
    
    from_to_form = (f"{from_state} -> {to_state}")
    
    return from_to_form

In [8]:
def comp_num_without_paranthesis(num):
    real_part = num.real
    im_part = num.imag
    return round(real_part,6), round(im_part,6)

In [9]:
def comp_coeff_parse(line):
    c = line.strip().split()[-1]
    # let's find the comma inside the string to distinguish the real and imaginary part of the coeff
    s = c.rfind(',')
    # the real part will be from 2nd char to just before comma and imag part starts just after comma
    # and ends on the last second character of the string
    real_c = float(c[1:s])
    im_c = float(c[s+1:-1])
    comp_c = complex(real_c, im_c)
    
    return comp_c

In [10]:
def prob_extraction(idx,pat2, all_lines):
    prob_list = []
    line = all_lines[idx]
    
    while not(re.match(pat2, line.strip())):
        prob = line.strip().split()[5]
        prob_list.append(prob)
        
        idx +=1
        line = all_lines[idx]
    
    prob_array = np.zeros(len(prob_list))
    prob_array[:] = prob_list
    
    return prob_array

In [11]:
def largest_ci_coeff_parsing(idx, pat2, all_lines):
    largest_coeffs = []
    from_to_states = []
    line = all_lines[idx]
        
    while not(re.match(pat2, line.strip())):
            
        comp_c = comp_coeff_parse(line)
        largest_coeffs.append(comp_c)
        from_to_state =from_to(line)
        from_to_states.append(from_to_state)
            
        idx +=  1
        line = all_lines[idx]
        
    return largest_coeffs, from_to_states, idx

In [12]:
num_roots = len(del_ex_root_ene)
for idx, line in enumerate(all_lines):
    if re.match(pat5, line.strip()):
        break

# let's go to first coeff line of Root 0--->
idx = idx+2
line = all_lines[idx]

file_write = open("ve.out", "w")

for n in range(num_roots):

    lc_list, from_to_states, updated_idx = largest_ci_coeff_parsing(idx, pat2, all_lines)

    # jumping to coefficient line for next root     
    idx = updated_idx + 3
    line = all_lines[idx]
    
    # changing format the largest coeff of current root from list to complex num array
    l = len(lc_list)
    coeffs_array = np.zeros(l, dtype = complex)
    coeffs_array[:] = lc_list[:]
    
    file_write.write(f"Root  {n+1}: Largest CI coefficients:\n")
    #print(f"Root  {n+1}: Largest CI coefficients:")
    for i in range(len(coeffs_array)):
        c_numb = coeffs_array[i]
        string_cnum = f"{c_numb}"
        c_numb = string_cnum[1:-1]
        #real, im = comp_num_without_paranthesis(coeffs_array[i])
        string = f"  {from_to_states[i]} : D -> V :  {c_numb}\n"
        #print(string)
        file_write.write(string)    
    file_write.write("\n")

#### So far we have succesfully found the delta excitation of all excited roots and largest ci coeff
#### Next task is to index the roots, find the g.s. energy,, oscillator stength and the max. CI coeff 

In [13]:
pat_gs = re.compile(r"Converged SCF results:")

In [14]:
for idx, line in enumerate(all_lines):
    if re.match(pat_gs, line.strip()):
        break
idx = idx + 9
line = all_lines[idx]

In [15]:
gs_ene = float(all_lines[idx].strip().split()[2])
gs_ene, del_ex_root_ene

(-174.6841952670456,
 [0.1744080457739137,
  0.2535162395738729,
  0.3483169930429212,
  0.3821066412378998,
  0.4030583583492398,
  0.486008501509091,
  0.5143698041793688,
  0.5893012916740668])

In [16]:
l = len(del_ex_root_ene)
del_ex_array = np.zeros(l)
del_ex_array[:] = del_ex_root_ene[:]

In [17]:
# broadcasting
ex_root_tot_ene = del_ex_array + gs_ene

 The idea is to make list of every column seperately, s.t. in every loop of i, we can call 
 the particluar elements
     For root number we can index from the one

                    Let's find the oscillator strength of length gauge

In [18]:
# We make use of double backslash to represent a literal paranthesis, 
# for regular expression to be able to read it as literal paranthesis, 
# also while compiling do not use 'r' in the front of string, otherwise re is not able to match 
pat_osc = re.compile("\\(length gauge\\):")

In [19]:
for idx, line in enumerate(all_lines):
    if re.match(pat_osc, line.strip()):
        break
osc_str_list = line.strip().split()[2: 2+num_roots]
osc_str_array = np.zeros(num_roots)
osc_str_array[:] = osc_str_list

                        For RHF, the S^2 remains zero

In [20]:
S_2_array = np.zeros(num_roots)

                Let's write the Final Excited state Results table

In [21]:
num_roots = len(del_ex_root_ene)
for idx, line in enumerate(all_lines):
    if re.match(pat5, line.strip()):
        break

# let's go to first coeff line of Root 0--->
idx = idx+2
line = all_lines[idx]

file_write.write("  Final Excited State Results:\n\n")
file_write.write("  Root   Total Energy (a.u.)   Ex. Energy (eV)   Osc. (a.u.)   <S^2>   Max CI Coeff.      Excitation\n")
file_write.write("------------------------------------------------------------------------------------------------------------\n")

for n in range(num_roots):
    lc_list, from_to_states, updated_idx = largest_ci_coeff_parsing(idx, pat2, all_lines)
    prob_array = prob_extraction(idx, pat2, all_lines)
    
    ind_maxprob = np.argmax(prob_array)
    #print(prob_array, ind_maxprob)
    
    # jumping to coefficient line for next root     
    idx = updated_idx + 3
    line = all_lines[idx]
    
    # changing format the largest coeff of current root from list to complex num array
    l = len(lc_list)
    coeffs_array = np.zeros(l, dtype = complex)
    coeffs_array[:] = lc_list[:]
    # Check for the largest coeff acc to the index of maxprob:
    max_cc = coeffs_array[ind_maxprob]
    string_max_cc = f"{max_cc}"
    st_max_cc = string_max_cc[1:-1]
    
    corresponding_ft_state = from_to_states[ind_maxprob]
    #print(corresponding_ft_state)
    
    file_write.write(f"     {n+1}        {round(ex_root_tot_ene[n],6)}        {round(del_ex_array[n]*27.211324570273,6)}       {round(osc_str_array[n],6)}      {S_2_array[n]}      {st_max_cc}     {corresponding_ft_state} : D -> V\n")
    
file_write.close()